<a href="https://colab.research.google.com/github/sushil79g/Nepali_text_summarizer/blob/master/Nepali_summerizer_tfidf.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#if you are using colab
!git clone https://github.com/sushil79g/Nepali_text_summerizer
from Nepali_text_summerizer.loadembedding import load_vector

Cloning into 'Nepali_text_summerizer'...
remote: Enumerating objects: 29, done.
remote: Counting objects: 100% (29/29), done.
remote: Compressing objects: 100% (27/27), done.
remote: Total 29 (delta 13), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (29/29), done.


In [0]:
#if you are at local
# from loadembedding import load_vector

In [2]:
word_vec = load_vector()

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:398: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


In [0]:
import string
def preprocess(text):
    sentences = text.split(u"।")
#     abc = text.translate(str.maketrans('', '', string.punctuation))
    sentences = [sentence.translate(str.maketrans('', '', string.punctuation)) for sentence in sentences]
    return sentences 

In [4]:
word_vec['याद']

array([ 1.59461051e-01, -5.85965931e-01,  3.78590047e-01, -4.74045545e-01,
        9.43699852e-02,  8.82810473e-01,  8.47471714e-01,  2.71366894e-01,
       -2.91820467e-01,  5.16441107e-01, -3.08772981e-01, -2.16735333e-01,
       -2.68063605e-01, -2.66613662e-01, -4.27117199e-01, -9.15338635e-01,
       -6.45890951e-01,  1.69385195e-01,  1.89498529e-01,  1.03605795e+00,
       -1.43938050e-01,  2.56531447e-01, -3.53469521e-01,  2.65364021e-01,
        3.85109991e-01,  3.01573038e-01,  7.09220588e-01, -6.60610273e-02,
        6.79583371e-01, -1.09058753e-01,  2.95875072e-01, -8.73443559e-02,
        1.30954489e-01,  4.19844911e-02,  4.76874322e-01,  5.39385259e-01,
        5.25668502e-01, -6.92974627e-02, -9.60046947e-02,  3.30410570e-01,
       -6.32967889e-01, -5.90312257e-02, -6.23273611e-01,  2.60242343e-01,
        5.40041029e-01,  2.43316591e-01,  2.72388130e-01,  4.72497255e-01,
       -1.95385918e-01, -8.09923261e-02, -8.45567584e-01, -1.58558071e-01,
       -7.79802799e-01, -

In [5]:
# (list(word_vec.vocab)) सिद्धान्त
word_vec.vector_size

400

In [0]:
#using TFIDF

In [0]:
from sklearn.feature_extraction.text import TfidfVectorizer
import numpy as np
def generate_centroif_tfidf(sentence):
    tf = TfidfVectorizer()
    tfidf = tf.fit_transform(sentence).toarray().sum(0)
    tfidf = np.divide(tfidf, tfidf.max())
    words = tf.get_feature_names()
    similar_term = []
    for i in range(len(tfidf)):
        if words[i] in vocab:
            if tfidf[i] >= 0.2:
                similar_term.append(words[i])
    res = [word_vec[term] for term in similar_term]
    return sum(res)/len(res)

In [0]:
vocab = word_vec.vocab
def sentence_vectorizer(sentence,size,word_vec,vocab):
    dic = {}
    for i in range(len(sentence)):
        sum_vec = np.zeros(size)
        try:
            sentence_word = sentence[i].split()
        except:
            pass
        sentence = [word for word in sentence_word if word in vocab]
        if sentence:
            for word in sentence:
                word_vec_ = word_vec[word]
                sum_vec = np.add(sum_vec, word_vec_)
                
            dic[i] = sum_vec/len(sentence)
    return dic


In [0]:
def sentence_selection(centroid, sentences_dict, summary_length):
    from scipy.spatial.distance import cosine
    sentence_retriever = []
    record = []
    for sentence_id in sentences_dict:
        vector = sentences_dict[sentence_id]
        similarity = (1 - cosine(centroid, vector))
        record.append((sentence_id, vector, similarity))

    rank = list(reversed(sorted(record, key=lambda tup: tup[2])))
    sentence_ids = []
    summary_char_num = 0
    stop = False
    i = 0
    text_length = sum([len(x) for x in sentence_retriever])

    if summary_length < 1:
        limit = int(text_length * float(summary_length))

        while not stop and i < len(rank):
            sentence_id = rank[i][0]
            new_vector = sentences_dict[sentence_id]
            sent_char_num = len(sentence_retriever[sentence_id])
            redundancy = [sentences_dict[k] for k in sentence_ids
                            if (1 - cosine(new_vector, sentences_dict[k]) > 0.95)]

            if not redundancy:
                summary_char_num += sent_char_num
                sentence_ids.append(sentence_id)
            i += 1

            if summary_char_num > limit:
                stop = True
    else: 
        sentences_number = int(summary_length)
        sentence_ids = rank[:sentences_number]
        sentence_ids = map(lambda t: t[0], sentence_ids)

    sentence_ids = sorted(sentence_ids)
    
    return sentence_ids

In [0]:
def summary(centroid_tfidf,sent,sent_dict,length):
    ids = sentence_selection(centroid_tfidf,sent_dict,length)
    whole_summary = []
    for inde in ids:
        whole_summary.append(sent[inde])
    return '।'.join(whole_summary)


In [0]:
sample_text = '''१७ साउन, काठमाडौं । स्थानीय राजनीतिकर्मी र स्वार्थ समूहको पेलाइका कारण चिनियाँ हुवाजिन कम्पनीले नेपालमा भित्र्याएको १४ अर्ब ४० करोड प्रत्यक्ष वैदेशिक लगानी संकटमा परेको छ । यही स्थिति जारी रहे सिमेन्ट उद्योगका लागि गरिएको सो लगानी फिर्ता लैजाने विषयमा समेत सो कम्पनीका अधिकारीहरु छलफलमा छन् ।चिनियाँ सरकारी स्वामित्वको कम्पनीले धादिङको बेनिघाट रोराङ गाउँपालिका-२ ताल्तीमा सिमेन्ट उद्योग निर्माणको काम गरिरहेको छ । तर, अनेक बहानामा काम प्रभावित भएपछि कम्पनी नेपालबाट फिर्ता हुन सक्ने अवस्था पैदा भएको स्रोतले बतायो । गत वर्ष असारमा प्रधानमन्त्री केपी शर्मा ओलीको चीन भ्रमणका क्रममा लगानी बोर्ड र हुवाजिनले सिमेन्ट उद्योगमा लगानी गर्ने सम्झौतामा हस्ताक्षर गरेका थिए ।कारखानासम्म जाने सडकको विषयमा सबैभन्दा ठूलो विवाद छ । शिलान्यास भएको ९ महिनामा उत्पादन थाल्ने लक्ष्य राखेको उद्योगले बेनीघाटबाट ११.५ किलोमिटर दुरीमा १२ मिटर चौडाइको सडक खन्न थालेको थियो । तर, यसमा छिमेकी गजुरी गाउँपालिकाका स्थानीयवासीले विरोध जनाए । कारखाना जाने बाटो र विद्युतसमेत आफ्नै गाउँबाट लैजानुपर्ने स्थानीयको माग छ । सोही गाउँपालिकाका प्रदेश सभा सांसद राजेन्द्र पाण्डेले असन्तुष्ट स्थानीयवासीको नेतृत्व गरेका छन् । गजुरीमा आयोजित एक कार्यक्रममा सांसद पाण्डेले सिमेन्ट उद्योगको डोजरमाथि आगो लगाइदिने सार्वजनिक भाषण गरेको बेनीघाट रोराङका स्थानीयले बताए । जबकी उद्योग शिलान्यासमा पाण्डे आफैं सहभागी थिए । उनै पाण्डे हालै स्थानीयवासीले बनाएको संघर्ष समितिको कार्यक्रममा सहभागी भएका थिए । यता, उद्योगले बनाइरहेको सडकले जैविक विविधता मासिएको तथा उद्योग सञ्चालनपछि वातावरण प्रदुषण फैलिने भन्दै एक संस्थाले विभिन्न निकायमा उजुरी गरेको छ । गजुरीका स्थानीय भूमि कंडेलकाे संयोजकत्वमा बनेको ‘वातावरण बचाउ अभियान’को उजुरीपछि वातावरण संरक्षणका क्षेत्रका कार्यरत विभिन्न संस्थाबाट स्थानीय सरकारलाई पत्र आएको बेनिघाट रोराङ गाउँपालिका-२ का वडा अध्यक्ष टंकबहादुर थापाले बताए । थापाका अनुसार वन मास्न नदिनू, खोला, कुलेसोमा सडक बनाउन नदिनू भन्ने खालका पत्रहरु वडामा आएका छन् । ओली गुटले धादिङमा पाण्डेलाई साइजमा राख्न मुख्यमन्त्रीको आश्वासनसहित प्रदेशसभामा पठाएर त्रिपाठीलाई प्रतिनिधिसभामा उम्मेदवार बनायो । त्यसयता पाण्डे समूह त्रिपाठी गुटविरुद्ध लागेको जनाकारहरु बताउँछन् । त्यसमाथि प्रदेश ३ को मुख्यमन्त्री बन्नबाट समेत पाण्डे बञ्चित गरिएपछि धादिङमा पाण्डेको राजनीतिक पकड त्रिपाठीतर्फ सरेको चर्चा व्यापक छ । प्रधानमन्त्री आफैंले भित्र्याएको चिनियाँ परियोजनामा त्रिपाठीले समन्वय गरिरहेका छन् भने व्यावसायी सुरज वैद्यले पनि १५ प्रतिशत लगानी गरेका छन् ।'''

In [0]:
def show_summary(sample_text,length_sentence_predict):
    sent = preprocess(sample_text)
    centroid_tfidf = generate_centroif_tfidf(sent)
    size = word_vec.vector_size
    sent_dict = sentence_vectorizer(sent,size,word_vec,vocab)
    return summary(centroid_tfidf,sent,sent_dict,length_sentence_predict)

In [12]:
from pprint import pprint
pprint('original')
pprint(sample_text)
print('\n\n')
pprint('summerized')
pprint(show_summary(sample_text,6))

'original'
('१७ साउन, काठमाडौं । स्थानीय राजनीतिकर्मी र स्वार्थ समूहको पेलाइका कारण '
 'चिनियाँ हुवाजिन कम्पनीले नेपालमा भित्र्याएको १४ अर्ब ४० करोड प्रत्यक्ष '
 'वैदेशिक लगानी संकटमा परेको छ । यही स्थिति जारी रहे सिमेन्ट उद्योगका लागि '
 'गरिएको सो लगानी फिर्ता लैजाने विषयमा समेत सो कम्पनीका अधिकारीहरु छलफलमा छन् '
 '।चिनियाँ सरकारी स्वामित्वको कम्पनीले धादिङको बेनिघाट रोराङ गाउँपालिका-२ '
 'ताल्तीमा सिमेन्ट उद्योग निर्माणको काम गरिरहेको छ । तर, अनेक बहानामा काम '
 'प्रभावित भएपछि कम्पनी नेपालबाट फिर्ता हुन सक्ने अवस्था पैदा भएको स्रोतले '
 'बतायो । गत वर्ष असारमा प्रधानमन्त्री केपी शर्मा ओलीको चीन भ्रमणका क्रममा '
 'लगानी बोर्ड र हुवाजिनले सिमेन्ट उद्योगमा लगानी गर्ने सम्झौतामा हस्ताक्षर '
 'गरेका थिए ।कारखानासम्म जाने सडकको विषयमा सबैभन्दा ठूलो विवाद छ । शिलान्यास '
 'भएको ९ महिनामा उत्पादन थाल्ने लक्ष्य राखेको उद्योगले बेनीघाटबाट ११.५ '
 'किलोमिटर दुरीमा १२ मिटर चौडाइको सडक खन्न थालेको थियो । तर, यसमा छिमेकी गजुरी '
 'गाउँपालिकाका स्थानीयवासीले विरोध जनाए । कारखाना जाने बाटो र विद्

In [0]:
from bs4 import BeautifulSoup
import urllib
import re
from urllib.request import Request, urlopen
def get_content(link):
  req = Request(link, headers={'User-Agent': 'Mozilla/5.0'})
  webpage = urlopen(req).read()
  soup = BeautifulSoup(webpage)
  text =''
  for tex_ in soup.find_all("p")[:-1]:
    text = text + " " + tex_.getText()
  return ''.join(text)
# sample_text = get_content('https://www.onlinekhabar.com/2019/08/789991')
# sample_text

In [32]:
!pip3 install news-please 
from newsplease import NewsPlease


     |████████████████████████████████| 71kB 2.8MB/s 
     |████████████████████████████████| 235kB 8.0MB/s 
     |████████████████████████████████| 51kB 23.5MB/s 
     |████████████████████████████████| 51kB 20.4MB/s 
     |████████████████████████████████| 143kB 35.5MB/s 
     |████████████████████████████████| 1.0MB 35.4MB/s 
     |████████████████████████████████| 51kB 21.1MB/s 
     |████████████████████████████████| 1.9MB 34.8MB/s 
     |████████████████████████████████| 215kB 36.3MB/s 
     |████████████████████████████████| 3.1MB 29.3MB/s 
     |████████████████████████████████| 61kB 20.3MB/s 
     |████████████████████████████████| 51kB 23.1MB/s 
     |████████████████████████████████| 5.7MB 31.1MB/s 
     |████████████████████████████████| 194kB 38.8MB/s 
     |████████████████████████████████| 7.4MB 21.5MB/s 
     |████████████████████████████████| 51kB 17.2MB/s 
     |████████████████████████████████| 174kB 36.6MB/s 
     |████████████████████████████████| 61kB 23.8MB/s 
  

In [0]:
def print_result(link):
    sample_text = get_content(link)
    article = NewsPlease.from_url(link)
    print('Title:',article.title)
    pprint('summerized')
    pprint(show_summary(sample_text,6))

In [40]:
print_result('https://www.onlinekhabar.com/2019/08/789118')

Title: विमानस्थलमा पारसको दुर्व्यवहार : प्रहरीले थाल्यो छानविन
'summerized'
(' २ भदौ काठमाडौं । मध्यप्रदेशको एन्टी टेरोरिज्म सेलले नक्कली भारतीय '
 'रुपैयाँसहित नेपाली नागरिक अतिक मोहम्मद र भारतीय नागरिक राजेश गुप्तालाई '
 'पक्राउ गरेपछि उनको संलग्नताबारे भारतीय सञ्चारमाध्यमले समाचार लेखेका थिए । '
 'त्यसवेला द इन्डिया टाइम्सले लेखेको एक समाचारमा शाहले अन्डरवर्ल्ड डन दाउद '
 'इब्राहिमको र्\u200dयाकेटमा नेपालबाट सहयोगीको रुपमा भूमिका खेलेको गम्भीर आरोप '
 'लगाइएको थियो । उक्त समाचारमा शाहसँगै युनुस अन्सारीलाई पनि भारतीय मिडियाले '
 'मुछेका थिए । तथापि पारशमाथि लागेको यो आरोपको पुष्टि हुन सकेको छैन ।')
